Note: This is (currently) the best code for identifying and analyzing clusters.

# A Possible Code for the Clusters

This code goes through the buffers one by one, rather than deconstructing all the data into one large array, like clusters_2.0 does.

First, we import Acqu (which parses the events?) and Timepix (which decodes the data??), as well as numpy, ROOT, and plotting classes of ROOT which allow us to organize and graph the data.

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas

Welcome to JupyROOT 6.16/00


Next, we import the data that was collected and open that data file.

In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Getting the data from detectors A and B:

In [3]:
TimepixAData = []
TimepixBData = []

TpxEvent = 0

def plotTimepix():
    global TpxEvent
    if(aq.epicsEvent==1):   # if there's some data (?)
        if(aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')!=TpxEvent):             # if this isn't a repeat of a previous buffer
            nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS') 
            encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
            encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
            TimepixAData.append(Timepix.Decode(nHitsA,encodedA))        # get data from timepix A for this buffer
            TimepixBData.append(Timepix.Decode(nHitsB,encodedB))        # get data from timepix B for this buffer
            TpxEvent = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')             # note the buffer we've gotten data from already

aq.runFunction(plotTimepix,0,50000)

A function that will help us sort the data by ToA:

In [4]:
def sortThird(val):     # a function to sort arrays of data by the third entry (in our case, ToA)
    return val[2]

A function that processes the clusters in detector A or detector B:

In [5]:
def process_cluster(let):
    start_index = cl[0]                     # start index of the first hit in the cluster
    min_time = buff[start_index]['ToA']     # time of the first hit in the cluster
    meanx = 0.0                             # mean x-position of the cluster
    meany = 0.0                             # mean y-position of the cluster
    meanToA = 0.0                           # mean ToA of the cluster
    
    n = start_index
    while(n<start_index+cn):                # go through each hit in the cluster
        if(buff[n]['ToA'] < min_time):      # if we find time smaller than the current minimum
            min_time = buff[n]['ToA']       # then reassign the smaller time to be the min time
        meanx+=buff[n]['x']                 # add up all the x-positions
        meany+=buff[n]['y']                 # add up all the y-positions
        meanToA+=buff[n]['ToA']             # add up all the ToAs
        n+=1                                # move to the next hit in the cluster
        
    meanx/=cn         # calculate the mean x-position
    meany/=cn         # calculate the mean y-position
    meanToA/=cn       # calculate the mean ToA
    
    # for detector A
    if(let=='a'):
        tdposA.Fill(meanx,meany)              # plot the mean y- vs. x-position of the cluster
        multA.Fill(cn)                        # plot the number of hits in the cluster
        mToAA.append(meanToA)                 # add the mean ToA of this cluster to the array

        n = start_index
        while(n<start_index+cn):                      # for each hit in the cluster
            dtime = buff[n]['ToA'] - min_time         # find the time difference between a hit in the cluster and the min time of the cluster
            dthistA.Fill(dtime)                       # plot the time difference
            dtime_toaA.Fill(dtime,buff[n]['ToT'])     # plot ToT vs. time difference
            n+=1                                      # move on to the next hit in the cluster
    
    # for detector B
    elif(let=='b'):
        tdposB.Fill(meanx,meany)              # plot the mean y- vs. x-position of the cluster
        multB.Fill(cn)                        # plot the number of hits in the cluster
        mToAB.append(meanToA)                 # add the mean ToA of this cluster to the array

        n = start_index
        while(n<start_index+cn):                      # for each hit in the cluster
            dtime = buff[n]['ToA'] - min_time         # find the time difference between a hit in the cluster and the min time of the cluster
            dthistB.Fill(dtime)                       # plot the time difference
            dtime_toaB.Fill(dtime,buff[n]['ToT'])     # plot ToT vs. time difference
            n+=1                                      # move on to the next hit in the cluster

A function for finding clusters in detector A or detector B:

In [6]:
def findclusters(let):
    
    global buff, cl, cn, blen, b, nc, mToAA, mToAB
    cl = [None]*100         # a cluster can hold up to 100 hits
    cn = 0                  # counts number of hits in the current cluster
    blen = 0                # the length of the current buffer
    b = 0                   # the running/current buffer position
    nc = 0                  # count the number of clusters found
    mToAA = []              # an array to hold the mean ToAs of the clusters for timepix A
    mToAB = []              # an array to hold the mean ToAs of the clusters for timepix B
    data = []  
    
    if(let=='a'):
        data = TimepixAData    # get the data for detector A
    elif(let=='b'):
        data = TimepixBData    # get the data for detector B
    
    for i in range(len(data)):           # for each buffer in the data
        b=0                              # reset to the start of the buffer
        buff = []                        # start with a fresh buffer
        for k in range(len(data[i])):    # for each hit in the buffer
            buff.append(data[i][k])      # put that hit in our current buffer
        buff.sort(key=sortThird)         # sort our current buffer by ToA
        blen = len(buff)                 # set blen to the length of our current buffer (i.e. the number of hits)
        
        while(b<blen):           # for every position in the buffer
            cn = 0               # reset the counter for hits in the cluster (we're starting a new cluster)
            cl = [None]*100      # reset the array of cluster hits (we're starting a new cluster)
            cl[cn] = b           # assume the current position has the first hit in our current cluster  
            cn+=1                # increment the number of hits for this cluster

            c=b+1       # start comparing at position c that is one above the current buffer position
            while((c<b+99) and (c<blen)):                         # check all positions up to 99 away, so long as we're within the buffer
                if(abs(buff[c]['ToA']- buff[b]['ToA'] < 100)):    # if a hit c is within 100ns of the current buffer position
                    cl[cn] = c                                    # save this position to the cluster
                    cn+=1                                         # increment the number of hits for this cluster
                else:
                    process_cluster(let)       # analyze the cluster we've found
                    b+=cn                      # move beyond this cluster
                    cn = 0                     # reset the counter for hits in the cluster (we're starting a new cluster)
                    cl = [None]*100            # reset the array of cluster hits
                    nc+=1                      # increment the number of clusters we found

                    if(c<b+99 and c<blen):     # if we're staring in the while loop after processing this cluster
                        cl[cn] = b             # assume the current position has the first hit in our new cluster
                        cn+=1                  # increment the number of hits for this cluster  
                        
                c+=1     # move to the next position for comparison
                
                if(((c>=blen) and cn!=0)or((c>=b+99) and cn!=0)):  # if exiting the loop but still have a cluster to process
                    process_cluster(let)                           # process the last cluster
                    b+=cn                                          # move beyond this cluster
                    cn = 0                                         # reset the counter for hits in the cluster
                    cl = [None]*100                                # reset the array of cluster hits
                    nc+=1                                          # increment the number of clusters we found
                    
            b+=1     # move to the next buffer position
            
        

Running and graphing the data for detector A:

In [7]:
ROOT.enableJSVis()
c1 = ROOT.TCanvas("c1","Clusters by Buffers", 1000, 2000)
c1.Divide(2,4)

# for detector A

tdposA = Hist2D(256,0,256,256,0,256)
tdposA.GetXaxis().SetTitle("mean x position of the cluster")
tdposA.GetYaxis().SetTitle("mean y position of the cluster")

multA = Hist(50,0,50)
multA.GetXaxis().SetTitle("number of hits in the cluster")
multA.GetYaxis().SetTitle("number of clusters")

dthistA = Hist(100,0,100)
dthistA.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dthistA.GetYaxis().SetTitle("number of clusters")

dtime_toaA = Hist2D(100,0,100,230,0,230)
dtime_toaA.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dtime_toaA.GetYaxis().SetTitle("ToT")

findclusters('a')

c1.cd(1)
tdposA.Draw("colz")
c1.cd(2)
multA.Draw()
c1.cd(3)
dthistA.Draw()
c1.cd(4)
dtime_toaA.Draw("colz")

# for detector B

tdposB = Hist2D(256,0,256,256,0,256)
tdposB.GetXaxis().SetTitle("mean x position of the cluster")
tdposB.GetYaxis().SetTitle("mean y position of the cluster")

multB = Hist(50,0,50)
multB.GetXaxis().SetTitle("number of hits in the cluster")
multB.GetYaxis().SetTitle("number of clusters")

dthistB = Hist(100,0,100)
dthistB.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dthistB.GetYaxis().SetTitle("number of clusters")

dtime_toaB = Hist2D(100,0,100,230,0,230)
dtime_toaB.GetXaxis().SetTitle("time diff between a hit and the min time of the cluster")
dtime_toaB.GetYaxis().SetTitle("ToT")

findclusters('b')

c1.cd(5)
tdposB.Draw("colz")
c1.cd(6)
multB.Draw()
c1.cd(7)
dthistB.Draw()
c1.cd(8)
dtime_toaB.Draw("colz")

c1.Draw()